In [ ]:
from src import *
from src import PickleManager
from src import Resolution
pm = PickleManager(directory="./pickle_data", verbose=True)

# Camera and scene setup

we can choose resolution of the camera from the predefined resolutions in the Resolution enum or make a custom one

In [ ]:
resolution = Resolution.R360p
#resolution = Resolution.custom(800, 600)

camera = Camera(
    fov=60,
    resolution=resolution,
    origin=Vertex(0, 0, 0),
    direction=Vector(0, 0, -1),
    up_hint=Vector(0, 1, 0)
)

print(f"Camera: {camera}")